# CaseStudy II
In this notebook, real-life MCDA problem will be solved using multiple modules. In this study case study we took 30 alternatives and use 3 criteria to sort alternatives into 3 categories using FlowSortI (M1).

## Definition of inputs and problem formalization

In [15]:
import pandas as pd
from core.enums import Direction, GeneralCriterion, SurrogateMethod, FlowType, CompareProfiles
from modular_parts.weights import surrogate_weights
from modular_parts.preference import (compute_preference_indices,
                                      compute_discordance)
from modular_parts.flows import calculate_promethee_outranking_flows
from modular_parts.sorting import calculate_flowsortI_sorted_alternatives


alternatives = ["LBS", "MIT", "YALE", "UNC", "TOR", "UCLA", "OXF", "MAR",
                "CAME",
                "CBS", "COR", "DUKE", "ESADE", "HARV", "IESA", "INSEAD",
                "NYU",
                "NWU", "ROTT", "STAN", "UCB", "CHI", "IOWA", "MICH", "PENN",
                "ROCH",
                "VIRG", "UWO", "VAND", "YORK"]
criteria = ['g1', 'g2', 'g3']
criteria_directions = [Direction.MAX, Direction.MAX, Direction.MAX]


generalised_criteria = [GeneralCriterion.V_SHAPE_INDIFFERENCE,
                        GeneralCriterion.V_SHAPE_INDIFFERENCE,
                        GeneralCriterion.V_SHAPE_INDIFFERENCE]
preference_thresholds = [3, 3, 3]
indifference_thresholds = [0.5, 0.5, 0.5]
alternatives_performances = [[68.78, 62.03, 59.87],
                             [60.01, 24.1, 85.81],
                             [79.01, 25.98, 51.84],
                             [67.8, 22.45, 62.4],
                             [59.34, 36.68, 60.79],
                             [55.46, 21.52, 74.54],
                             [57.04, 43.49, 47.12],
                             [55.33, 25.28, 64.13],
                             [54.02, 18.69, 71.93],
                             [83.44, 32.94, 71.63],
                             [60.54, 30.31, 55.99],
                             [64.05, 27.25, 64.68],
                             [54.61, 60.15, 14.45],
                             [77.84, 29.05, 93.91],
                             [61.99, 54.61, 40.04],
                             [56.59, 70.74, 65.45],
                             [68.24, 26.74, 80.4],
                             [68.68, 24.52, 72.43],
                             [51.2, 52.91, 42.98],
                             [76.77, 28.52, 81.8],
                             [61.26, 31.04, 73.69],
                             [75.64, 23.57, 77.73],
                             [58.25, 26.25, 46.81],
                             [63.23, 28.36, 69.61],
                             [77.11, 31.51, 91.59],
                             [60.25, 29.6, 49.25],
                             [76.49, 19.05, 37.68],
                             [50.04, 40.55, 53.55],
                             [64.67, 26.33, 35.14],
                             [57.03, 50.73, 47.55],
                             ]

criteria_directions = pd.Series(criteria_directions, criteria)
generalised_criteria = pd.Series(generalised_criteria, criteria)
preference_thresholds = pd.Series(preference_thresholds, criteria)
indifference_thresholds = pd.Series(indifference_thresholds, criteria)

alternatives_performances = pd.DataFrame(data=alternatives_performances,
                                         index=alternatives, columns=criteria)
alternatives_performances

,g1,g2,g3
LBS,68.78,62.03,59.87
MIT,60.01,24.10,85.81
YALE,79.01,25.98,51.84
UNC,67.80,22.45,62.40
TOR,59.34,36.68,60.79
UCLA,55.46,21.52,74.54
OXF,57.04,43.49,47.12
MAR,55.33,25.28,64.13
CAME,54.02,18.69,71.93
CBS,83.44,32.94,71.63


In [16]:
categories = [f"C{i}" for i in range(1, 4)]
profiles = [f"r{i}" for i in range(1, 4)]
category_profiles_performances = pd.DataFrame(
    [[56, 25, 48], [68, 38, 70], [75, 45, 85]],
    index=profiles, columns=criteria)
category_profiles_performances

,g1,g2,g3
r1,56,25,48
r2,68,38,70
r3,75,45,85


## Modules usage

Weights calculated with Reciprocal of Ranks method.

In [17]:
criteria_weights = surrogate_weights(pd.Series([1, 3, 2], criteria),
                                     SurrogateMethod.RR)
criteria_weights

g1    0.545
g2    0.182
g3    0.273
Name: weights, dtype: float64

Promethee Preferences need to be calculated twice: for alternatives vs alternatives and profiles vs profiles

In [18]:
preference_indices, partial_preference_indices = compute_preference_indices(
    alternatives_performances, preference_thresholds, indifference_thresholds,
    pd.Series(data=None, index=criteria), generalised_criteria,
    criteria_directions, criteria_weights, category_profiles_performances)
partial_preference_indices

C:\Users\teba_sgrabowski\AppData\Local\Temp\ipykernel_17776\623476402.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series(data=None, index=criteria), generalised_criteria,


(                r1     r2   r3
 criteria                      
 g1       LBS   1.0  0.112  0.0
          MIT   1.0  0.000  0.0
          YALE  1.0  1.000  1.0
          UNC   1.0  0.000  0.0
          TOR   1.0  0.000  0.0
 ...            ...    ...  ...
 g3       ROCH  0.3  0.000  0.0
          VIRG  0.0  0.000  0.0
          UWO   1.0  0.000  0.0
          VAND  0.0  0.000  0.0
          YORK  0.0  0.000  0.0
 
 [90 rows x 3 columns],
              LBS   MIT  YALE   UNC    TOR   UCLA    OXF    MAR   CAME  CBS  \
 criteria                                                                     
 g1       r1  0.0  0.00   0.0  0.00  0.000  0.016  0.000  0.068  0.592  0.0   
          r2  0.0  1.00   0.0  0.00  1.000  1.000  1.000  1.000  1.000  0.0   
          r3  1.0  1.00   0.0  1.00  1.000  1.000  1.000  1.000  1.000  0.0   
 g2       r1  0.0  0.16   0.0  0.82  0.000  1.000  0.000  0.000  1.000  0.0   
          r2  0.0  1.00   1.0  1.00  0.328  1.000  0.000  1.000  1.000  1.0   
     

In [19]:
preference_indices

(           r1     r2     r3
 LBS     1.000  0.243  0.182
 MIT     0.818  0.273  0.034
 YALE    0.853  0.545  0.545
 UNC     0.818  0.000  0.000
 TOR     1.000  0.000  0.000
 UCLA    0.273  0.273  0.000
 OXF     0.300  0.182  0.000
 MAR     0.273  0.000  0.000
 CAME    0.273  0.156  0.000
 CBS     1.000  0.668  0.545
 COR     1.000  0.000  0.000
 DUKE    0.945  0.000  0.000
 ESADE   0.182  0.182  0.182
 HARV    1.000  0.818  0.783
 IESA    0.727  0.182  0.182
 INSEAD  0.475  0.182  0.182
 NYU     0.908  0.273  0.000
 NWU     0.818  0.250  0.000
 ROTT    0.182  0.182  0.182
 STAN    1.000  0.818  0.277
 UCB     1.000  0.273  0.000
 CHI     0.818  0.818  0.031
 IOWA    0.436  0.000  0.000
 MICH    1.000  0.000  0.000
 PENN    1.000  0.818  0.624
 ROCH    0.809  0.000  0.000
 VIRG    0.545  0.545  0.216
 UWO     0.455  0.149  0.000
 VAND    0.605  0.000  0.000
 YORK    0.298  0.182  0.182,
       LBS    MIT   YALE    UNC    TOR   UCLA    OXF    MAR   CAME    CBS  ...  \
 r1  0.000  0.029 

In [20]:
preference_indices_profiles, partial_preference_indices_profiles = compute_preference_indices(
    category_profiles_performances, preference_thresholds,
    indifference_thresholds,
    pd.Series(data=None, index=criteria), generalised_criteria,
    criteria_directions, criteria_weights)

partial_preference_indices_profiles

C:\Users\teba_sgrabowski\AppData\Local\Temp\ipykernel_17776\3409108458.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series(data=None, index=criteria), generalised_criteria,


r1   r2   r3
criteria                  
g1       r1  0.0  0.0  0.0
         r2  1.0  0.0  0.0
         r3  1.0  1.0  0.0
g2       r1  0.0  0.0  0.0
         r2  1.0  0.0  0.0
         r3  1.0  1.0  0.0
g3       r1  0.0  0.0  0.0
         r2  1.0  0.0  0.0
         r3  1.0  1.0  0.0

In [21]:
preference_indices_profiles

,r1,r2,r3
r1,0.0,0.0,0.0
r2,1.0,0.0,0.0
r3,1.0,1.0,0.0


We additionally wanted to include not only the good but also the bad performance on the criteria, so we calculated the discordance indices using the Promethee Discordance (M6). Module M6 is also used twice: for alternatives vs profiles preference and for profiles' preference.

In [22]:
tau = 1
discordance, partial_discordance, overall_preference = compute_discordance(
    criteria,
    partial_preference_indices, tau,
    preferences=preference_indices,
    were_categories_profiles=True)
partial_discordance

[          r1   r2   r3
 g1 LBS   0.0  0.0  1.0
    MIT   0.0  1.0  1.0
    YALE  0.0  0.0  0.0
    UNC   0.0  0.0  1.0
    TOR   0.0  1.0  1.0
 ...      ...  ...  ...
 g3 ROCH  0.0  1.0  1.0
    VIRG  1.0  1.0  1.0
    UWO   0.0  1.0  1.0
    VAND  1.0  1.0  1.0
    YORK  0.0  1.0  1.0
 
 [90 rows x 3 columns],
          LBS    MIT   YALE  UNC  TOR  UCLA    OXF  MAR   CAME    CBS  ...  \
 g1 r1  1.000  1.000  1.000  1.0  1.0   0.0  0.216  0.0  0.000  1.000  ...   
    r2  0.112  0.000  1.000  0.0  0.0   0.0  0.000  0.0  0.000  1.000  ...   
    r3  0.000  0.000  1.000  0.0  0.0   0.0  0.000  0.0  0.000  1.000  ...   
 g2 r1  1.000  0.000  0.192  0.0  1.0   0.0  1.000  0.0  0.000  1.000  ...   
    r2  1.000  0.000  0.000  0.0  0.0   0.0  1.000  0.0  0.000  0.000  ...   
    r3  1.000  0.000  0.000  0.0  0.0   0.0  0.000  0.0  0.000  0.000  ...   
 g3 r1  1.000  1.000  1.000  1.0  1.0   1.0  0.000  1.0  1.000  1.000  ...   
    r2  0.000  1.000  0.000  0.0  0.0   1.0  0.000  0.0  0.572

In [23]:
discordance

[           r1   r2   r3
 LBS     0.000  1.0  1.0
 MIT     0.056  1.0  1.0
 YALE    0.000  1.0  1.0
 UNC     0.435  1.0  1.0
 TOR     0.000  1.0  1.0
 UCLA    1.000  1.0  1.0
 OXF     0.053  1.0  1.0
 MAR     0.023  1.0  1.0
 CAME    1.000  1.0  1.0
 CBS     0.000  1.0  1.0
 COR     0.000  1.0  1.0
 DUKE    0.000  1.0  1.0
 ESADE   1.000  1.0  1.0
 HARV    0.000  1.0  1.0
 IESA    1.000  1.0  1.0
 INSEAD  0.000  1.0  1.0
 NYU     0.000  1.0  1.0
 NWU     0.000  1.0  1.0
 ROTT    1.000  1.0  1.0
 STAN    0.000  1.0  1.0
 UCB     0.000  1.0  1.0
 CHI     0.144  1.0  1.0
 IOWA    0.102  1.0  1.0
 MICH    0.000  1.0  1.0
 PENN    0.000  1.0  1.0
 ROCH    0.000  1.0  1.0
 VIRG    1.000  1.0  1.0
 UWO     1.000  1.0  1.0
 VAND    1.000  1.0  1.0
 YORK    0.000  1.0  1.0,
     LBS    MIT  YALE  UNC  TOR  UCLA  OXF  MAR   CAME  CBS  ...  UCB    CHI  \
 r1  1.0  1.000   1.0  1.0  1.0   1.0  1.0  1.0  1.000  1.0  ...  1.0  1.000   
 r2  1.0  1.000   1.0  0.0  0.0   1.0  1.0  0.0  0.246  1.0  ...

In [24]:
overall_preference

(           r1   r2   r3
 LBS     1.000  0.0  0.0
 MIT     0.772  0.0  0.0
 YALE    0.853  0.0  0.0
 UNC     0.462  0.0  0.0
 TOR     1.000  0.0  0.0
 UCLA    0.000  0.0  0.0
 OXF     0.284  0.0  0.0
 MAR     0.267  0.0  0.0
 CAME    0.000  0.0  0.0
 CBS     1.000  0.0  0.0
 COR     1.000  0.0  0.0
 DUKE    0.945  0.0  0.0
 ESADE   0.000  0.0  0.0
 HARV    1.000  0.0  0.0
 IESA    0.000  0.0  0.0
 INSEAD  0.475  0.0  0.0
 NYU     0.908  0.0  0.0
 NWU     0.818  0.0  0.0
 ROTT    0.000  0.0  0.0
 STAN    1.000  0.0  0.0
 UCB     1.000  0.0  0.0
 CHI     0.700  0.0  0.0
 IOWA    0.392  0.0  0.0
 MICH    1.000  0.0  0.0
 PENN    1.000  0.0  0.0
 ROCH    0.809  0.0  0.0
 VIRG    0.000  0.0  0.0
 UWO     0.000  0.0  0.0
 VAND    0.000  0.0  0.0
 YORK    0.298  0.0  0.0,
     LBS    MIT  YALE    UNC    TOR  UCLA    OXF  MAR   CAME  CBS  ...  UCB  \
 r1  0.0  0.000   0.0  0.000  0.000   0.0  0.000  0.0  0.000  0.0  ...  0.0   
 r2  0.0  0.000   0.0  0.455  0.878   0.0  0.000  1.0  0.548  0.0 

In [25]:
discordance_prof, partial_discordance_prof, overall_preference_prof = compute_discordance(
    criteria,
    partial_preference_indices_profiles, tau,
    preferences=preference_indices_profiles,
    were_categories_profiles=False)
overall_preference_prof

,r1,r2,r3
r1,0.0,0.0,0.0
r2,1.0,0.0,0.0
r3,1.0,1.0,0.0


Given the preference degrees, we can compute outranking flows for both alternatives and profiles.

In [26]:
flows_alter = calculate_promethee_outranking_flows(overall_preference,
                                                   FlowType.BASIC)
flows_alter

,positive,negative
LBS,0.333333,0.000000
MIT,0.257333,0.232000
YALE,0.284333,0.000000
UNC,0.154000,0.485000
TOR,0.333333,0.626000
UCLA,0.000000,0.333333
OXF,0.094667,0.297333
MAR,0.089000,0.666667
CAME,0.000000,0.516000
CBS,0.333333,0.000000


In [27]:
flows_profiles = calculate_promethee_outranking_flows(overall_preference_prof,
                                                      FlowType.BASIC)
flows_profiles

,positive,negative
r1,0.0,1.0
r2,0.5,0.5
r3,1.0,0.0


Finally, we sorted alternatives using FlowSortI method (M20).

In [28]:
calculate_flowsortI_sorted_alternatives(categories,
                                                     category_profiles_performances,
                                                     criteria_directions,
                                                     flows_alter,
                                                     flows_profiles,
                                                     CompareProfiles.CENTRAL_PROFILES)

,worse,better
LBS,C2,C3
MIT,C2,C3
YALE,C2,C3
UNC,C1,C2
TOR,C2,C2
UCLA,C1,C2
OXF,C1,C2
MAR,C1,C2
CAME,C1,C2
CBS,C2,C3
